In [180]:
import pandas as pd
import matplotlib.pylab as plt
from datetime import datetime, timezone, timedelta

plot_data = {}

data = pd.read_csv("data/dialogs/594508393.csv")

def tsv_to_dict(tsv_path="tone-dict-uk.tsv"):
    
    
    sentiment_data = pd.read_csv("tone-dict-uk.tsv", sep="\t")
    sentiment_dict = dict(zip(sentiment_data.iloc[:, 0], sentiment_data.iloc[:, 1]))

    return sentiment_dict 
    
def is_date_after(check_date, days_ago = 365):

    date_today = datetime.now(timezone.utc)
    days_ago = timedelta(days = days_ago)
    final_date = date_today - days_ago
    if check_date > final_date:
        return True
    return False

def divide_and_prepare_data(data, days_ago = 365):
    
    prepped_data = {}
    for index, row in data.iterrows():
        
        msg_date = datetime.fromisoformat(row['date'])
        if not is_date_after(msg_date, days_ago):
            break
        if not row['from_id'] in prepped_data:
            prepped_data[row['from_id']] = {
                day_num : set() for day_num in range(1, 8)
            }
        
        msg_week_day = msg_date.isoweekday()
        prepped_data[row['from_id']][msg_week_day].add(row['message'])
        
    return prepped_data
    
def sentiment_text_analysis(data, sentiment_dict_path="tone-dict-uk.tsv"):
    
    sentiment_result = {}
    text_data = divide_and_prepare_data(data)
    word_dict = tsv_to_dict(sentiment_dict_path)
    for user_id in text_data:
        
        if not user_id in sentiment_result:
            sentiment_result[user_id] = {
                day_num : 0 for day_num in range(1, 8)
            }
        
        for day_number in text_data[user_id]:
            
            score = 0
            
            while text_data[user_id][day_number]:
                
                msg = text_data[user_id][day_number].pop()

                if isinstance(msg, str):
                    tokens = msg.strip().split()
                    for token in tokens:
                        if token.lower() in word_dict:
                            score += word_dict[token.lower()]
                 
            sentiment_result[user_id][day_number] = score
            
            
    return sentiment_result
    
res = sentiment_text_analysis(data)
print(res)
def plot_sentiment_analysis_results(result_data):
    
    for user_id, sentiment_info in result_data.items():
        print(sentiment_info)



plot_sentiment_analysis_results(res)




{594508393: {1: -6, 2: -1, 3: 11, 4: -6, 5: -3, 6: 3, 7: -1}, 418590848: {1: -4, 2: -7, 3: -5, 4: -4, 5: -8, 6: -3, 7: -9}}
{1: -6, 2: -1, 3: 11, 4: -6, 5: -3, 6: 3, 7: -1}
{1: -4, 2: -7, 3: -5, 4: -4, 5: -8, 6: -3, 7: -9}


In [32]:
# Counting messages throughout hours.

for x in data.index:
    if not plot_data.get(data['date'][x][11:13]):
        plot_data.setdefault(data['date'][x][11:13], 1)
    else:
        plot_data[data['date'][x][11:13]] += 1

In [ ]:
# Ploting data

lists = sorted(plot_data.items())
x, y = zip(*lists)


plt.plot(x, y)
plt.xlabel('Hours')
plt.ylabel('Number of messages overall')
plt.show()





In [ ]:
from messages_per_hours